# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846753238711                   -0.70    4.2
  2   -7.852263983678       -2.26       -1.53    1.0
  3   -7.852645951639       -3.42       -2.53    3.0
  4   -7.852646676246       -6.14       -3.35    2.5
  5   -7.852646685929       -8.01       -4.74    1.2
  6   -7.852646686725       -9.10       -5.25    4.2
  7   -7.852646686729      -11.38       -5.87    2.0
  8   -7.852646686730      -12.38       -7.26    2.0
  9   -7.852646686730      -14.27       -7.77    2.5
 10   -7.852646686730   +  -14.45       -8.94    2.0
 11   -7.852646686730      -14.75       -9.50    1.8
 12   -7.852646686730   +    -Inf       -8.70    1.0
 13   -7.852646686730   +  -14.27       -8.83    1.2
 14   -7.852646686730      -14.27       -8.51    2.0
 15   -7.852646686730   +  -14.57       -9.04    1.0
 16   -7.852646686730   +    -Inf       -8.89    1.0
 17   -7.852646686730      -15.05       -9.72 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846815799667                   -0.70           4.8
  2   -7.852525955694       -2.24       -1.63   0.80    2.0
  3   -7.852637155634       -3.95       -2.70   0.80    1.0
  4   -7.852646403098       -5.03       -3.31   0.80    2.2
  5   -7.852646682103       -6.55       -4.12   0.80    1.8
  6   -7.852646686383       -8.37       -4.73   0.80    1.2
  7   -7.852646686722       -9.47       -5.76   0.80    1.5
  8   -7.852646686730      -11.10       -6.49   0.80    2.0
  9   -7.852646686730      -12.73       -7.40   0.80    1.8
 10   -7.852646686730      -14.21       -8.16   0.80    2.5
 11   -7.852646686730      -14.45       -8.29   0.80    2.0
 12   -7.852646686730   +  -14.45       -9.32   0.80    1.0
 13   -7.852646686730      -14.45      -10.03   0.80    2.2
 14   -7.852646686730   +  -14.75      -10.33   0.80    2.0
 15   -7.852646686730   +    -Inf      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.398401e+01     3.840823e+00
 * time: 0.35711216926574707
     1     1.188877e+00     1.730289e+00
 * time: 0.5120270252227783
     2    -1.279008e+00     2.235397e+00
 * time: 0.5306510925292969
     3    -3.783845e+00     1.801057e+00
 * time: 0.55715012550354
     4    -4.908720e+00     1.466029e+00
 * time: 0.5837070941925049
     5    -6.735595e+00     1.135926e+00
 * time: 0.6111931800842285
     6    -7.434914e+00     3.339177e-01
 * time: 0.6379101276397705
     7    -7.661337e+00     1.585963e-01
 * time: 0.6571481227874756
     8    -7.747480e+00     1.414059e-01
 * time: 0.6766271591186523
     9    -7.803402e+00     7.817952e-02
 * time: 0.6954360008239746
    10    -7.829947e+00     7.455909e-02
 * time: 0.714846134185791
    11    -7.839078e+00     7.032872e-02
 * time: 0.7344551086425781
    12    -7.844134e+00     5.183019e-02
 * time: 0.7541611194610596
    13    -7.847935e+00     3.699192e-02
 * time: 0.773428201675

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846795706654                   -0.70    4.8
  2   -7.852316138705       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686705                   -2.54
  2   -7.852646686730      -10.61       -5.91
  3   -7.852646686730   +  -15.05      -12.71


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.4013864271517352e-13
|ρ_newton - ρ_scfv| = 5.421598366710728e-13
|ρ_newton - ρ_dm|   = 8.446781017588987e-10
